In [1]:
# !pip uninstall etl_pipeline -y

In [2]:
# !pip install --no-deps etl_pipeline-0.5.16.dev0-py3-none-any.whl

In [1]:
import os
import glob
import warnings


os.environ["CONFIG_APP_DIR"] = "../ms_debugging/config/"
# os.environ["CONFIG_APP_DIR"] = "20220331_1506config/"

In [2]:
import sys
import pickle
from omegaconf import OmegaConf
from etl_pipeline.config import pipeline_config
from etl_pipeline.custom.ms.payload_loader import PayloadLoader

import json
from etl_pipeline.data_processor_engine.json_engine.json_engine import JsonProcessingEngine

/env/ds/anaconda/envs/pipeline/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from pipelines.ms.ms_pipeline import MSPipeline, pipeline_config
cn = pipeline_config.cn

In [4]:
ETL_SAMPLES_PATH = '/v/region/na/appl/itlnc/s8/data/uat/silent8/name-screening/etl_samples/complete_info/'

In [5]:
DATASET_NAME = 'WM' # 'ISG_Daily' # 'WM' # 'ISG_Weekly 
DATASET_TYPE =  'Party' # 'Address' # Party # Account
DATASET = DATASET_NAME + '_' + DATASET_TYPE

In [6]:
datasets = ['ISG_Daily_Party', 'ISG_Daily_Account', 'ISG_Weekly_Party', 'ISG_Weekly_Account', 'WM_Party', 'WM_Address']
complete_info = {dataset: glob.glob(ETL_SAMPLES_PATH + dataset + '/*.json') for dataset in datasets}

JSON_DIR = f'notebooks/application/processed/{DATASET}'
%mkdir -p {JSON_DIR}

### Testing flow

In [7]:
from pipelines.ms.ms_pipeline import MSPipeline

engine = JsonProcessingEngine(pipeline_config)
pipeline = MSPipeline(engine, config=pipeline_config)

Field in agent configuration for: ALL_CONCAT_NAMES is not registered field in pipeline.yaml


In [10]:
with open(f'sample/wm_party_in_payload_format.json', 'r') as file:
    payload = json.loads(file.read())

payload_json = {key: payload[key] for key in sorted(payload)}
payload_json = PayloadLoader().load_payload_from_json(payload_json)


payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"][cn.MATCH_RECORDS]))]


In [11]:
payload = pipeline.transform_standardized_to_cleansed(payload_json)
new_payloads = pipeline.transform_cleansed_to_application(payload)

In [12]:
import pandas as pd

In [13]:
out_payload = pd.DataFrame([match for payload in new_payloads   for match in payload['watchlistParty']['matchRecords']])

In [14]:
out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,ap_all_nationalities_aggregated,wl_all_nationalities_aggregated,ap_all_party_types_aggregated,wl_all_party_types_aggregated,ap_all_pobs_aggregated,wl_all_pobs_aggregated,ap_all_santioned_countries_aggregated,wl_all_santioned_countries_aggregated,ap_all_residencies_aggregated,wl_all_residencies_aggregated,ap_all_hit_types_aggregated,wl_all_hit_types_aggregated
0,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]","[022368917, 154421273]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],[],[Joe Ding],[United States],[],[Individual],[I],[],[Joe Ding],[],[Joe Ding],[United States],[],[],[]
1,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]","[022368917, 154421273]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Dong],[],[Joe Dong],[United States],"[PL, Poland]",[Individual],[I],[],"[PL, Joe Dong, Poland]",[],"[PL, Joe Dong, Poland]",[United States],"[PL, Poland]",[],[]


# run etl

In [15]:
source_files = ['sample/wm_party_in_payload_format.json', 'sample/wm_address_in_payload_format.json']

In [16]:
from tqdm import tqdm
from dataclasses import asdict, is_dataclass
from joblib import Parallel, delayed


def dump_application(source_files, target_path="notebooks/application/processed", batch_size=10000, n_jobs=4):
    def process_file(info):
        with open(info, 'r') as file:
            payload = json.load(file)

        payload_json = {key: payload[key] for key in sorted(payload)}
        payload_json = PayloadLoader().load_payload_from_json(payload_json)
        payload_json['match_ids'] = [i for i in range(len(payload_json["watchlistParty"][cn.MATCH_RECORDS]))]
        payload = pipeline.transform_standardized_to_cleansed(payload_json)
        new_payloads = pipeline.transform_cleansed_to_application(payload)

        for i in new_payloads:
            for m in i['alertedParty']['inputRecordHist']['inputRecords']:
                for key in m['INPUT_FIELD']:
                    if is_dataclass(m['INPUT_FIELD'][key]):
                        m['INPUT_FIELD'][key] = asdict(m['INPUT_FIELD'][key])
                    elif isinstance(m['INPUT_FIELD'][key], dict):
                        continue
                    else:
                        raise TypeError('field type not implemented')
        return (info, new_payloads)
    
    def save_file(info, new_payloads):
        with open(os.path.join(target_path, "application_" + os.path.basename(info)), 'w') as file:
            json.dump(new_payloads, file) 
    
    os.makedirs(target_path, exist_ok=True)
    
    counter = 0
    no_of_chunks = 1 + len(source_files) // batch_size
    for chunk in (source_files[i: i+batch_size] for i in range(0, len(source_files), batch_size)):
        # separated in-memory jobs from I/O to enable proper parallelisation
        counter += 1
        print(f'processing chunk {counter} of {no_of_chunks}')
        payloads_chunk = Parallel(n_jobs=n_jobs)(delayed(process_file)(info)for info in tqdm(source_files))
        print('saving')
        Parallel(n_jobs=n_jobs)(delayed(save_file)(*entry) for entry in tqdm(payloads_chunk))

In [17]:
for dataset in datasets:
    print(dataset)
    JSON_DIR = f'notebooks/application/processed/{dataset}'
    !mkdir -p {JSON_DIR}
    dump_application(complete_info[dataset][:2000], JSON_DIR)

ISG_Daily_Party


  0%|          | 0/2000 [00:00<?, ?it/s]

processing chunk 1 of 1


  6%|▌         | 120/2000 [00:00<00:01, 1076.67it/s]

saving


...

  0%|          | 0/2000 [00:00<?, ?it/s]

processing chunk 1 of 1


  6%|▌         | 120/2000 [00:00<00:01, 1164.34it/s]

saving


100%|██████████| 2000/2000 [00:02<00:00, 765.72it/s]


In [18]:
processed = glob.glob(f'notebooks/application/processed/WM_Party/*.json')
with open(processed[0], 'r')as file:
    print(file.read())

[{"alertedParty": {"supplementalInfo": {"datasetName": "R_US_Active_Party", "alertId": "SANC-ASM-7211412", "uniqueCustId": "R_US_Active_Party_0053610458", "busDate": "11/27/2021", "screeningId": null, "datasetId": null, "masterId": null, "relatedParties": {"party": [{"partyPrimaryCitizenshipCountry": "United States", "partyId": "0053610458", "partyCountryOfBirth": null, "partyName": "Lanter, Hope Danielle", "partyMiddleName": "Danielle", "partyLastName": "Lanter", "countryOfIncorporation": null, "taxId": "240270025", "partyType": "Individual", "taxIdType": "SSN", "dobDate": "12/02/1977", "partyFirstName": "Hope", "partyResidenceCountry": "United States", "partyGovtIdType": null, "partyGovtIdNumber": null, "partySecondaryCitizenshipCountry": null, "accountBU": null, "CONNECTED_FULL_NAME": "Hope Danielle Lanter"}]}, "relatedAccounts": {"accounts": [{"lastName": "Lanter", "branchAccountNumber": "405_306012", "srcSystemAccountKey": "2021-05-06-12.52.57.091945", "firstName": "Hope", "benefi

In [19]:
new_payloads

[{'alertedParty': {'headerInfo': {'currentVersionId': '122438658',
    'datasetId': '1044',
    'datasetName': 'R_US_Active_Party',
    'firstVersionCreatedDt': '2020-01-08T09:31:30.264-05:00',
    'inputVersionSample': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.',
    'lastVersionUpdatedDt': '2020-01-08T09:31:30.264-05:00',
    'masterId': '72951854',
    'masterVersion': '412740c151535f0756e8dbec7440b726c7a3e135',
    'stopDescriptors': {'stopDescriptor': {'name': 'Joe Doe'}},
    'uniqueCustomerId': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
   'inputRecordHist': {'inputRecords': [{'createdDate': '01/08/20',
      'fields': [{'isScreenable': 'false',
        'name': 'SOURCE_REF',
        'sortOrder': '1',
        'value': 'R_US_Active_Address_A05003324172_2020-01-07-07.06.28.836480'},
       {'isScreenable': 'false',
        'name': 'UNIQUE_KEY',
        'sortOrder': '2',
        'value': 'A05003324172'},
       {'isScreenable': 'false', 'name': 'BARCODE

In [20]:
out_payload = pd.DataFrame([match for payload in new_payloads for match in payload['watchlistParty']['matchRecords']])

In [21]:
out_payload[[i for i in out_payload.columns  if i.endswith("_aggregated")]]

,ap_all_dobs_aggregated,wl_all_dobs_aggregated,ap_all_documents_aggregated,wl_all_documents_aggregated,ap_all_names_aggregated,wl_all_names_aggregated,ap_all_employer_names_aggregated,wl_all_employer_names_aggregated,ap_all_nationalities_aggregated,wl_all_nationalities_aggregated,ap_all_party_types_aggregated,wl_all_party_types_aggregated,ap_all_pobs_aggregated,wl_all_pobs_aggregated,ap_all_santioned_countries_aggregated,wl_all_santioned_countries_aggregated,ap_all_residencies_aggregated,wl_all_residencies_aggregated,ap_all_hit_types_aggregated,wl_all_hit_types_aggregated
0,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]","[022368917, 154421273]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Ding],[],[Joe Ding],[United States],[],[Individual],[I],[],[Joe Ding],[],[Joe Ding],[United States],[],[],[]
1,"[04/31/1910, 02/31/1900]","[MAY 6, 1981]","[022368917, 154421273]",[],"[Eva Pladimirova, Pladimir Vutin]",[Joe Dong],[],[Joe Dong],[United States],"[PL, Poland]",[Individual],[I],[],"[PL, Joe Dong, Poland]",[],"[PL, Joe Dong, Poland]",[United States],"[PL, Poland]",[],[]
